In [1]:
from pyspark.sql import SparkSession;

# warehouse_location points to the default location for managed databases and tables
from os.path import abspath
warehouse_location = abspath('spark-warehouse')

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("ISM6562 PySpark Tutorials") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()


# Let's get the SparkContext object. It's the entry point to the Spark API. It's created when you create a sparksession
sc = spark.sparkContext

# note: If you have multiple spark sessions running (like from a previous notebook you've run), 
# this spark session webUI will be on a different port than the default (4040). One way to 
# identify this part is with the following line. If there was only one spark session running, 
# this will be 4040. If it's higher, it means there are still other spark sesssions still running.
spark_session_port = spark.sparkContext.uiWebUrl.split(":")[-1]
print("Spark Session WebUI Port: " + spark_session_port)

23/10/26 15:18:26 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 10.21.5.100 instead (on interface eth0)
23/10/26 15:18:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/26 15:18:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session WebUI Port: 4040


In [2]:
spark

In [3]:
spark.sql("show tables").show()

23/10/26 15:18:30 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/10/26 15:18:30 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/10/26 15:18:33 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/10/26 15:18:33 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore student@127.0.0.1
23/10/26 15:18:33 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|  default|fake_friends|      false|
|  default|   incidents|      false|
|  default|movieratings|      false|
|  default|      movies|      false|
+---------+------------+-----------+



In [4]:
df = spark.sql("select * from incidents")

In [5]:
df.printSchema()

root
 |-- number: string (nullable = true)
 |-- incident_state: string (nullable = true)
 |-- active: boolean (nullable = true)
 |-- reassignment_count: integer (nullable = true)
 |-- reopen_count: integer (nullable = true)
 |-- sys_mod_count: integer (nullable = true)
 |-- made_sla: boolean (nullable = true)
 |-- caller_id: string (nullable = true)
 |-- opened_by: string (nullable = true)
 |-- opened_at: string (nullable = true)
 |-- sys_created_by: string (nullable = true)
 |-- sys_created_at: string (nullable = true)
 |-- sys_updated_by: string (nullable = true)
 |-- sys_updated_at: string (nullable = true)
 |-- contact_type: string (nullable = true)
 |-- location: string (nullable = true)
 |-- category: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- u_symptom: string (nullable = true)
 |-- cmdb_ci: string (nullable = true)
 |-- impact: string (nullable = true)
 |-- urgency: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- assignment_gr

In [6]:
df_unique_incidents=df.filter("incident_state=='Closed'").sort("sys_mod_count",ascending=False).dropDuplicates(["number"])

In [7]:
### 1. Top 5 people with most resolved incidents

In [8]:
A1=df_unique_incidents.groupby("resolved_by").count().sort("count",ascending=False)

A1.show(n=5)

+---------------+-----+
|    resolved_by|count|
+---------------+-----+
| Resolved by 11| 3071|
| Resolved by 15| 2415|
|Resolved by 103|  689|
|Resolved by 177|  686|
| Resolved by 32|  597|
+---------------+-----+
only showing top 5 rows



### 2. Based on least average duration, find the top 5 people with maxmium number of incidents resolved

In [9]:
from pyspark.sql import functions as F

A2= df_unique_incidents.groupby("resolved_by").agg(F.count("duration"),F.mean("duration")).\
    withColumnRenamed("count(duration)","Incidents Resolved").\
    withColumnRenamed("avg(duration)","Average Duration").\
    orderBy(["Average Duration","Incidents Resolved"],ascending=[True,False])

#spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")
A2.show(n=5)

+---------------+------------------+----------------+
|    resolved_by|Incidents Resolved|Average Duration|
+---------------+------------------+----------------+
| Resolved by 10|                 4|             0.0|
| Resolved by 94|                 4|             0.0|
| Resolved by 26|                 2|             0.0|
|Resolved by 145|                 2|             0.0|
| Resolved by 17|                 1|             0.0|
+---------------+------------------+----------------+
only showing top 5 rows



### 3. People with maximum number of high impact incidents resolved

In [10]:
A3= df_unique_incidents.select(["resolved_by","impact","duration"]).\
    groupby(["impact","resolved_by"]).count().\
    sort(["impact","count"],ascending=[True,False])

A3.show()

+--------+---------------+-----+
|  impact|    resolved_by|count|
+--------+---------------+-----+
|1 - High| Resolved by 98|   20|
|1 - High|Resolved by 137|   17|
|1 - High| Resolved by 11|   15|
|1 - High|Resolved by 165|   13|
|1 - High|  Resolved by 6|   12|
|1 - High|Resolved by 111|   12|
|1 - High|Resolved by 223|   10|
|1 - High|Resolved by 158|    9|
|1 - High| Resolved by 91|    9|
|1 - High|Resolved by 139|    9|
|1 - High|Resolved by 150|    9|
|1 - High| Resolved by 90|    8|
|1 - High|Resolved by 119|    8|
|1 - High|Resolved by 155|    8|
|1 - High| Resolved by 23|    7|
|1 - High| Resolved by 15|    7|
|1 - High| Resolved by 19|    6|
|1 - High| Resolved by 83|    6|
|1 - High| Resolved by 57|    6|
|1 - High| Resolved by 77|    6|
+--------+---------------+-----+
only showing top 20 rows



### 4a. In each impact levels, find the person with most number of incidents resolve

In [11]:
A4a=df_unique_incidents.select(["resolved_by","impact","duration"]).\
    groupby(["impact","resolved_by"]).count().\
    sort(["impact","count"],ascending=[True,False]).\
    dropDuplicates(["impact"])

A4a.show()

+----------+--------------+-----+
|    impact|   resolved_by|count|
+----------+--------------+-----+
|  1 - High|Resolved by 98|   20|
|2 - Medium|Resolved by 11| 3045|
|   3 - Low|Resolved by 66|  194|
+----------+--------------+-----+



### 4b. In each urgency levels, find the person with most number of incidents resolved

In [12]:
A4b=df_unique_incidents.select(["resolved_by","urgency","duration"]).\
    groupby(["urgency","resolved_by"]).count().\
    sort(["urgency","count"],ascending=[True,False]).dropDuplicates(["urgency"])

A4b.show()

+----------+---------------+-----+
|   urgency|    resolved_by|count|
+----------+---------------+-----+
|  1 - High|Resolved by 166|   38|
|2 - Medium| Resolved by 11| 3047|
|   3 - Low| Resolved by 66|  195|
+----------+---------------+-----+



### 4c. In each priority levels, find the person with most number of incidents resolved

In [13]:
A4c=df_unique_incidents.select(["resolved_by","priority","duration"]).\
    groupby(["priority","resolved_by"]).count().\
    sort(["priority","count"],ascending=[True,False]).dropDuplicates(["priority"])

A4c.show()

+------------+---------------+-----+
|    priority|    resolved_by|count|
+------------+---------------+-----+
|1 - Critical| Resolved by 98|   16|
|    2 - High|Resolved by 166|   40|
|3 - Moderate| Resolved by 11| 3040|
|     4 - Low| Resolved by 66|  195|
+------------+---------------+-----+



### 5. Find each contact type as a percentage of total incidents

In [14]:
from pyspark.sql.window import Window

A5= df_unique_incidents.select(["contact_type"]).\
    groupby(["contact_type"]).count().\
    withColumn("percentage",F.round(F.col("count")*100/F.sum("count").over(Window.partitionBy()),2))

A5.show()

23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 15:18:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/10/26 1

+--------------+-----+----------+
|  contact_type|count|percentage|
+--------------+-----+----------+
|         Phone|24688|     99.08|
|         Email|   59|      0.24|
|  Self service|  158|      0.63|
|           IVR|    9|      0.04|
|Direct opening|    4|      0.02|
+--------------+-----+----------+



### 6. On each priority level, find the percentage of incidents which made SLA and which did not.

In [15]:
A6= df_unique_incidents.select(["priority","made_sla"]).\
    groupby(["priority","made_sla"]).count().\
    withColumnRenamed("count","Population").\
    withColumn("Made SLA %",F.round(F.col("Population")*100/F.sum("Population").over(Window.partitionBy("priority")),2)).\
    sort(["priority","made_sla"],ascending=[True,False])

A6.show()

+------------+--------+----------+----------+
|    priority|made_sla|Population|Made SLA %|
+------------+--------+----------+----------+
|1 - Critical|    true|         5|      1.85|
|1 - Critical|   false|       265|     98.15|
|    2 - High|    true|         2|      0.49|
|    2 - High|   false|       406|     99.51|
|3 - Moderate|    true|     15145|     64.54|
|3 - Moderate|   false|      8321|     35.46|
|     4 - Low|    true|       651|     84.11|
|     4 - Low|   false|       123|     15.89|
+------------+--------+----------+----------+



### 7. Top 5 location with the maximum number of incidents reported

In [16]:
A7= df_unique_incidents.groupby(["location"]).agg({"number":"count"}).\
    withColumnRenamed("count(number)","Incidents Reported").sort(["Incidents Reported"],ascending=False)

A7.show()

+------------+------------------+
|    location|Incidents Reported|
+------------+------------------+
|Location 204|              5554|
|Location 161|              4002|
|Location 143|              3276|
|Location 108|              2140|
| Location 93|              1934|
| Location 51|              1516|
|Location 188|               622|
|Location 179|               346|
| Location 43|               329|
|Location 111|               264|
| Location 55|               241|
| Location 46|               221|
|Location 125|               216|
| Location 96|               174|
|Location 135|               172|
| Location 54|               154|
| Location 56|               150|
|Location 229|               122|
| Location 42|               119|
| Location 97|               113|
+------------+------------------+
only showing top 20 rows



### 8. Which category of issues missed meeting the SLA the most?

In [17]:
A8= df_unique_incidents.filter("made_sla==false").groupby(["category"]).\
    agg({"made_sla":"count"}).withColumnRenamed("count(made_sla)","Incidents failed to make SLA").\
    sort(["Incidents failed to make SLA"],ascending=False)

A8.show()

+-----------+----------------------------+
|   category|Incidents failed to make SLA|
+-----------+----------------------------+
|Category 46|                        1254|
|Category 26|                        1017|
|Category 53|                        1009|
|Category 42|                         689|
|Category 23|                         505|
| Category 9|                         471|
|Category 45|                         446|
|Category 61|                         429|
|Category 57|                         390|
|Category 37|                         385|
|Category 32|                         377|
|Category 34|                         333|
|Category 24|                         303|
|Category 40|                         266|
|Category 20|                         241|
|Category 28|                         160|
|Category 44|                         151|
|Category 19|                         130|
|Category 51|                         109|
|Category 55|                          84|
+----------

In [18]:
spark.stop()